<center><img src="media/imagenes/Banner.png" height = 100 width = 1000></center>

In [1]:
from sys import implementation

import dependencias
from pprint import pprint

pygame 2.6.1 (SDL 2.28.4, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Reglas

Hay que enseñar lo que hemos implentado del juego base

## Agentes

### Agentes Basados en Heuristicas.

Hemos definido un primer tipo de agente basado en heurísticas que se basa en los siguientes pasos:

- get_reward: Calcula la recompensa para cada movimiento en funcion de una politica de recompensa.
- get_action: Calcula la mejor acción a realizar en función de la recompensa obtenida.

de este modo podemos empezar a jugar con el agente y ver como se comporta.

## Agentes Chaser

La politica de recompensa de este agente se basa en la distancia entre la cabeza de la serpiente y la comida, de este modo el agente siempre intentará acercarse a la comida, aunque esto implique que la serpiente se muera.

In [3]:
agente_chaser = dependencias.Agentes.ChaserAgent()

In [15]:
# Ejecuta esta celda para ver como se comporta el agente
game = dependencias.Snake_game((15, 15), 5, agente_chaser)
game.play_with_pygame()

In [23]:
estadisticas = game.evaluar()
pprint(estadisticas)

100%|██████████| 100/100 [00:00<00:00, 1775.93it/s]

{'movimientos por puntuacion': np.float64(4.296235679214402),
 'movimientos_maximos': np.int64(115),
 'movimientos_medios': np.float64(26.25),
 'movimientos_minimos': np.int64(5),
 'puntuacion_maxima': np.int64(23),
 'puntuacion_media': np.float64(6.11),
 'puntuacion_minima': np.int64(3)}


### Conclusiones

Este agente tarda solo 4 movimientos en conseguir la comida, pero no alcanza puntuaciones muy altas.

### Mejoras

Este agente podría ser mejorado si se le añade una penalización por muerte inmediata.

## Agentes Avoider

La politica de recompensa de este agente se basa en evitar las muertes inediatas, ya sea contra la serpriente o contra las paredes, asociando una penalización a estos movimientos.

In [4]:
agente_avoider = dependencias.Agentes.Avoid_inmediate_death()

In [26]:
# Ejecuta esta celda para ver como se comporta el agente
game = dependencias.Snake_game((15, 15), 5, agente_avoider)
game.play_with_pygame()

In [28]:
estadisticas = game.evaluar()
pprint(estadisticas)

100%|██████████| 100/100 [00:00<00:00, 421.66it/s]

{'movimientos por puntuacion': np.float64(59.01053740779768),
 'movimientos_maximos': np.int64(1268),
 'movimientos_medios': np.float64(560.01),
 'movimientos_minimos': np.int64(91),
 'puntuacion_maxima': np.int64(17),
 'puntuacion_media': np.float64(9.49),
 'puntuacion_minima': np.int64(4)}


### Conclusiones

Este agente majora mucho su tiempo de vida, pero no consigue alcanzar la comida de manera eficiente.

### Mejoras

Ademas vemos que con evitar muertes inmediatas no es suficiente, puesto que hay escenarios en los que te puedes quedar encerrado.

## Combinacion de agentes

Esta implementation de los agentes tan particular, nos perite combinar las recompensas que le dan a cada movimiento distintos agentes para conseguir un agente que se comporte de una manera más eficiente.

In [3]:
agente_combinado = dependencias.Agentes.Combined_agent((agente_avoider, agente_chaser), (1, 0.5))

NameError: name 'agente_chaser' is not defined

In [5]:
# Ejecuta esta celda para ver como se comporta el agente
game = dependencias.Snake_game((15, 15), 5, agente_combinado)
game.play_with_pygame()

In [31]:
estadisticas = game.evaluar()
pprint(estadisticas)

100%|██████████| 100/100 [00:00<00:00, 440.01it/s]

{'movimientos por puntuacion': np.float64(60.81171067738231),
 'movimientos_maximos': np.int64(2610),
 'movimientos_medios': np.float64(529.67),
 'movimientos_minimos': np.int64(133),
 'puntuacion_maxima': np.int64(16),
 'puntuacion_media': np.float64(8.71),
 'puntuacion_minima': np.int64(4)}


### Conclusiones

El comportaiento de este agente es mucho mejor que el de los agentes individuales, y parece una buena estrategia para el inicio del juego.

### Mejoras

El agente sigue sin ser capaz de evitar quedarse encerrado, hay que añadir una heuristica mas a futuro para evitar este tipo de situaciones.

In [15]:
agente_searcher = dependencias.Agentes.Aantiloop()
agente_avoider = dependencias.Agentes.Avoid_inmediate_death()
agente_chaser = dependencias.Agentes.ChaserAgent()

agente_combinado = dependencias.Agentes.Combined_agent((agente_avoider, agente_chaser, agente_searcher), (1, 0, 10))
game = dependencias.Snake_game((15, 15), 5, agente_combinado)
game.play_with_pygame()

In [17]:
estadisticas = game.evaluar(10)
pprint(estadisticas)

100%|██████████| 10/10 [00:34<00:00,  3.41s/it]

{'movimientos por puntuacion': np.float64(40.07934508816121),
 'movimientos_maximos': np.int64(3628),
 'movimientos_medios': np.float64(3182.3),
 'movimientos_minimos': np.int64(2509),
 'puntuacion_maxima': np.int64(98),
 'puntuacion_media': np.float64(79.4),
 'puntuacion_minima': np.int64(66)}


In [21]:
agente_combinado = dependencias.Agentes.Combined_agent((agente_avoider, agente_chaser, agente_searcher), (1, 0.2, 3))
game = dependencias.Snake_game((15, 15), 5, agente_combinado)
game.play_with_pygame()

In [24]:
estadisticas = game.evaluar(100)
pprint(estadisticas)

100%|██████████| 100/100 [00:57<00:00,  1.74it/s]

{'movimientos por puntuacion': np.float64(6.8849013073570875),
 'movimientos_maximos': np.int64(971),
 'movimientos_medios': np.float64(537.16),
 'movimientos_minimos': np.int64(294),
 'puntuacion_maxima': np.int64(126),
 'puntuacion_media': np.float64(78.02),
 'puntuacion_minima': np.int64(48)}
